<a href="https://colab.research.google.com/github/yashrajOjha/multilingual-ner/blob/main/XLM_Roberta_for_Multilang_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!nvidia-smi

Sun Feb 26 13:47:39 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P0    26W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install numpy seqeval pandas matplotlib scikit-learn transformers datasets

In [5]:
import torch

if torch.cuda.is_available():
    print("GPU is enabled.")
    print("device count: {}, current device: {}".format(torch.cuda.device_count(), torch.cuda.current_device()))
else:
    print("GPU is not enabled.")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

GPU is enabled.
device count: 1, current device: 0


In [6]:
from datasets import load_dataset
languages=['bn','hi', 'kn', 'mr','ta', 'te']
hin_dataset = load_dataset('ai4bharat/naamapadam',languages[1],split='train[:10%]')
kan_dataset = load_dataset('ai4bharat/naamapadam',languages[2],split='train[:10%]')
ben_dataset = load_dataset('ai4bharat/naamapadam',languages[0],split='train[:10%]')
mar_dataset = load_dataset('ai4bharat/naamapadam',languages[3],split='train[:10%]')
tam_dataset = load_dataset('ai4bharat/naamapadam',languages[4],split='train[:10%]')
tel_dataset = load_dataset('ai4bharat/naamapadam',languages[5],split='train[:10%]')

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset naamapadam downloaded and prepared to /root/.cache/huggingface/datasets/ai4bharat___naamapadam/hi/1.0.0/c1b045180d60b208d2468bdad897d04461f08c7137c04a85220697b1bef7df9a. Subsequent calls will reuse this data.


In [16]:
print("hindi dataset",hin_dataset)
print("kannada dataset",kan_dataset)
print("bengali dataset",ben_dataset)
print("marathi dataset",mar_dataset)
print("tamil dataset",tam_dataset)
print("telugu dataset",tel_dataset)

hindi dataset Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 98579
})
kannada dataset Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 47176
})
bengali dataset Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 96168
})
marathi dataset Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 45525
})
tamil dataset Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 49788
})
telugu dataset Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 50774
})


In [19]:
hin_dataset_val = load_dataset('ai4bharat/naamapadam',languages[1],split='validation[:10%]')
kan_dataset_val = load_dataset('ai4bharat/naamapadam',languages[2],split='validation[:10%]')
ben_dataset_val = load_dataset('ai4bharat/naamapadam',languages[0],split='validation[:10%]')
mar_dataset_val = load_dataset('ai4bharat/naamapadam',languages[3],split='validation[:10%]')
tam_dataset_val = load_dataset('ai4bharat/naamapadam',languages[4],split='validation[:10%]')
tel_dataset_val = load_dataset('ai4bharat/naamapadam',languages[5],split='validation[:10%]')

In [20]:
print("hindi dataset validation",hin_dataset_val)
print("kannada dataset validation",kan_dataset_val)
print("bengali dataset validation",ben_dataset_val)
print("marathi dataset validation",mar_dataset_val)
print("tamil dataset validation",tam_dataset_val)
print("telugu dataset validation",tel_dataset_val)

hindi dataset validation Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 1346
})
kannada dataset validation Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 238
})
bengali dataset validation Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 486
})
marathi dataset validation Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 230
})
tamil dataset validation Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 280
})
telugu dataset validation Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 270
})


In [21]:
print(languages)

['bn', 'hi', 'kn', 'mr', 'ta', 'te']


In [24]:
raw_dataset = {}
raw_dataset['hi'] = hin_dataset
raw_dataset['kn'] = kan_dataset
raw_dataset['bn'] = ben_dataset
raw_dataset['mr'] = mar_dataset
raw_dataset['ta'] = tam_dataset
raw_dataset['te'] = tel_dataset

raw_dataset_val = {}
raw_dataset_val['hi'] = hin_dataset_val
raw_dataset_val['kn'] = kan_dataset_val
raw_dataset_val['bn'] = ben_dataset_val
raw_dataset_val['mr'] = mar_dataset_val
raw_dataset_val['ta'] = tam_dataset_val
raw_dataset_val['te'] = tel_dataset_val

In [25]:
pre_concatenated_train_split = []

for key,val in raw_dataset.items():
  pre_concatenated_train_split.append(val)

pre_concatenated_validation_split = []

for key,val in raw_dataset_val.items():
  pre_concatenated_validation_split.append(val)

In [26]:
from datasets import concatenate_datasets, DatasetDict

dataset = DatasetDict()
dataset["train"] = concatenate_datasets(
    pre_concatenated_train_split
)
dataset["validation"] = concatenate_datasets(
    pre_concatenated_validation_split
)

In [28]:
print("train dataset",dataset["train"])
print("validation dataset",dataset["validation"])

train dataset Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 388010
})
validation dataset Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 2850
})


In [29]:
tags = dataset["train"].features["ner_tags"].feature
index2tag = {idx: tag for idx, tag in enumerate(tags.names)}
tag2index = {tag: idx for idx, tag in enumerate(tags.names)}
tag2index

{'O': 0,
 'B-PER': 1,
 'I-PER': 2,
 'B-ORG': 3,
 'I-ORG': 4,
 'B-LOC': 5,
 'I-LOC': 6}

In [30]:
num_labels = len(tag2index)

In [31]:
from transformers import AutoTokenizer
xlmr_tokenizer = AutoTokenizer.from_pretrained("cfilt/HiNER-original-xlm-roberta-large")
print("The maximum length for the inputs is {}".format(xlmr_tokenizer.model_max_length))

The maximum length for the inputs is 512


In [32]:
# Tokenize all texts and align the labels with them.
padding = "max_length"
text_column_name = 'tokens'
label_column_name = 'ner_tags'
def tokenize_and_align_labels(examples):
    tokenized_inputs = xlmr_tokenizer(
        examples[text_column_name],
        padding=padding,
        truncation=True,
        max_length=512,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [34]:
tokenized_ds = dataset["train"].map(
    tokenize_and_align_labels,
    remove_columns=["tokens", "ner_tags"],
    batched=True,
    batch_size=256)

Map:   0%|          | 0/388010 [00:00<?, ? examples/s]

In [35]:
len(tokenized_ds)

388010

In [36]:
val_tokenized = dataset["validation"].map(
    tokenize_and_align_labels,
    remove_columns=["tokens", "ner_tags"],
    batched=True,
    batch_size=128)

Map:   0%|          | 0/2850 [00:00<?, ? examples/s]

In [37]:
# pd.DataFrame(
#     [xlmr_tokenizer.convert_ids_to_tokens(tokenized_ds[0]["input_ids"]), tokenized_ds[0]["labels"]],
#     index=["tokens", "ner_tags"])

for ind, val in enumerate(xlmr_tokenizer.convert_ids_to_tokens(tokenized_ds[0]["input_ids"])):
  print(val,tokenized_ds[0]["labels"][ind])

<s> -100
▁से 0
क्टर -100
▁55 0
/ -100
56 -100
▁के 0
▁एस 0
एच -100
ओ -100
▁अरविंद 1
▁कुमार 2
▁ने 0
▁बताया 0
▁कि 0
▁इस 0
▁मामले 0
▁में 0
▁आई 0
पी -100
सी -100
▁की 0
▁धारा 0
▁3 0
76 -100
▁- 0
▁डी 0
▁( 0
▁ग 0
ै -100
ंग -100
रे -100
प -100
▁) 0
▁के 0
▁तहत 0
▁मामला 0
▁दर्ज 0
▁कर 0
▁लिया 0
▁गया 0
▁है 0
▁। 0
</s> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> 

In [39]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir = "xlm-roberta-ner",
    log_level = "error",
    num_train_epochs = 1,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    evaluation_strategy = "steps",
    eval_steps = 500, # Evaluation and Save happens every 10 steps
    fp16 = True,
    logging_steps = len(tokenized_ds),
    save_total_limit=4,
    load_best_model_at_end= True,
    push_to_hub = False
    )

In [40]:
%env TOKENIZERS_PARALLELISM=false

env: TOKENIZERS_PARALLELISM=false


In [41]:
import numpy as np
# see https://huggingface.co/spaces/evaluate-metric/seqeval
from seqeval.metrics import f1_score

def metrics_func(eval_arg):
    preds = np.argmax(eval_arg.predictions, axis=2)
    batch_size, seq_len = preds.shape
    y_true, y_pred = [], []
    for b in range(batch_size):
        true_label, pred_label = [], []
        for s in range(seq_len):
            if eval_arg.label_ids[b, s] != -100:  # -100 must be ignored
                true_label.append(index2tag[eval_arg.label_ids[b][s]])
                pred_label.append(index2tag[preds[b][s]])
        y_true.append(true_label)
        y_pred.append(pred_label)
    return {"f1": f1_score(y_true, y_pred)}

In [42]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(
    xlmr_tokenizer,
    return_tensors="pt")

In [43]:
from transformers import AutoConfig, Trainer
from transformers.models.roberta.modeling_roberta import RobertaForTokenClassification

xlmr_config = AutoConfig.from_pretrained(
    "xlm-roberta-base",
    num_labels=tags.num_classes,
    id2label=index2tag,
    label2id=tag2index
)
model = (RobertaForTokenClassification
         .from_pretrained("xlm-roberta-base", config=xlmr_config)
         .to(device))

trainer = Trainer(
    model = model,
    args = training_args,
    data_collator = data_collator,
    compute_metrics = metrics_func,
    train_dataset = tokenized_ds,
    eval_dataset = val_tokenized,
    tokenizer = xlmr_tokenizer
)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing RobertaForTokenClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to

In [ ]:
import torch
torch.cuda.empty_cache()

In [44]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,F1
500,No log,0.270799,0.685052
1000,No log,0.265068,0.689560
1500,No log,0.256571,0.705681
2000,No log,0.249606,0.711365
2500,No log,0.251647,0.710548
3000,No log,0.246587,0.713013
3500,No log,0.241985,0.716203
4000,No log,0.308881,0.673232
4500,No log,0.235672,0.714440
5000,No log,0.234695,0.722392


TrainOutput(global_step=24251, training_loss=0.2298870649019628, metrics={'train_runtime': 18628.4286, 'train_samples_per_second': 20.829, 'train_steps_per_second': 1.302, 'total_flos': 1.0139033769317376e+17, 'train_loss': 0.2298870649019628, 'epoch': 1.0})

In [45]:
import os

os.makedirs("./xlm-roberta-mner", exist_ok=True)
#trainer.save_model("./trained_ner_classifier_jp")
if hasattr(trainer.model, "module"):
    trainer.model.module.save_pretrained("./xlm-roberta-mner")
else:
    trainer.model.save_pretrained("./xlm-roberta-mner")

In [46]:
!zip -r /content/xlm-roberta-mner.zip /content/xlm-roberta-mner

  adding: content/xlm-roberta-mner/ (stored 0%)
  adding: content/xlm-roberta-mner/pytorch_model.bin (deflated 27%)
  adding: content/xlm-roberta-mner/config.json (deflated 53%)


In [49]:
import torch
from transformers import AutoConfig

xlmr_config = AutoConfig.from_pretrained(
    "xlm-roberta-base",
    num_labels=tags.num_classes,
    id2label=index2tag,
    label2id=tag2index
)
model = (RobertaForTokenClassification
         .from_pretrained("./xlm-roberta-mner", config=xlmr_config)
         .to(device))

In [57]:
from datasets import Dataset
import torch
from torch.utils.data import DataLoader
import pandas as pd

sample_encoding = xlmr_tokenizer([
   "ನಗರದ ಯಲಹಂಕ ಏರ್​ ಬೇಸ್​ನಲ್ಲಿ ನಡೆಯುವ ಏರೋ ಇಂಡಿಯಾ ಏರ್​​ ಶೋ ಉದ್ಘಾಟನೆ ಸಂಬಂಧ ಪ್ರಧಾನಿ ನರೇಂದ್ರ ಮೋದಿ ಬೆಂಗಳೂರಿಗೆ ಬಂದಿಳಿದಿದ್ದಾರೆ",
   "ಯಶರಾಜ್ ನಾಳೆ ಗೋರಖ್‌ಪುರಕ್ಕೆ ಹೋಗುತ್ತಿದ್ದಾರೆ, ಅದು ಭಾರತದ ನಗರ",
   "যশরাজ নেদারল্যান্ডসে জাতিসংঘে যাচ্ছেন",
   "यशराज नेदरलँडमध्ये असलेल्या युनायटेड नेशन्समध्ये जाणार आहे",
   "யாஷ்ராஜ் நெதர்லாந்தில் உள்ள UNO விற்கு செல்கிறார்",
   "యష్‌రాజ్ నెదర్లాండ్స్‌లోని ఐక్యరాజ్యసమితికి వెళ్తున్నారు"
  # "सेक्टर 55/56 क एसएचओ अरविंद कुमार ने बताया कि इस मामले"
], truncation=True, max_length=512)
sample_dataset = Dataset.from_dict(sample_encoding)
sample_dataset = sample_dataset.with_format("torch")

sample_dataloader = DataLoader(sample_dataset, batch_size=1)
tokens = []
labels = []
for batch in sample_dataloader:
    # predict
    with torch.no_grad():
        output = model(batch["input_ids"].to(device), batch["attention_mask"].to(device))
    predicted_label_id = torch.argmax(output.logits, axis=-1).cpu().numpy()
    # create output
    tokens.append(xlmr_tokenizer.convert_ids_to_tokens(batch["input_ids"][0]))
    labels.append([index2tag[i] for i in predicted_label_id[0]])

pd.DataFrame([tokens[5], labels[5]], index=["Tokens", "Tags"])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Tokens,<s>,▁య,ష్,▁రాజ్,▁నె,దర్,లాండ్,స్,▁లోని,▁ఐ,క్,య,రాజ్య,స,మి,తికి,▁వెళ్త,ు,న్నారు,</s>
Tags,O,B-PER,B-PER,I-PER,B-LOC,B-LOC,B-LOC,B-LOC,O,B-ORG,B-ORG,B-ORG,I-ORG,I-ORG,I-ORG,O,O,O,O,O


In [47]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [48]:
!cp /content/xlm-roberta-mner.zip '/content/gdrive/MyDrive/multilingual-ner/'
!ls -lt '/content/gdrive/MyDrive/multilingual-ner/' 

total 787765
-rw------- 1 root root 806670708 Feb 26 19:42 xlm-roberta-mner.zip
